In [1]:
from bokeh.plotting import figure, save, show, output_file, output_notebook
from bokeh.models import GeoJSONDataSource, ColumnDataSource, Range1d, HoverTool, LogColorMapper, BoxZoomTool
from bokeh.resources import CDN
from bokeh.layouts import row
from bokeh.palettes import Blues6 as palette
from bokeh.models.widgets import RadioButtonGroup
from bokeh.models.callbacks import CustomJS
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("ex.xlsx", encoding='utf-8')
#PATH = r"mapdata_t/TOWN_MOI_1071031.shp"
PATH = r"mapdata_c/COUNTY_MOI_1070516.shp"
gdf = gpd.read_file(PATH, encoding='utf-8') #全台灣村里界圖
gdf = gdf.to_crs({'init' :'epsg:4326'})
palette.reverse()

In [3]:
gdf.columns

Index(['COUNTYID', 'COUNTYCODE', 'COUNTYNAME', 'COUNTYENG', 'geometry'], dtype='object')

In [4]:
gdf.head()

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry
0,Z,09007,連江縣,Lienchiang County,(POLYGON ((119.9644519140001 25.94552317400007...
1,G,10002,宜蘭縣,Yilan County,(POLYGON ((121.9597084550001 24.84493697000005...
2,N,10007,彰化縣,Changhua County,"POLYGON ((120.4565526600001 24.20718620500008,..."
3,M,10008,南投縣,Nantou County,"POLYGON ((121.2708644380001 24.23660985400005,..."
4,P,10009,雲林縣,Yunlin County,"(POLYGON ((120.081077282 23.52412216400006, 12..."


In [5]:
ndf = pd.merge(gdf, df)
villages_shp = gpd.GeoDataFrame(ndf)

In [6]:
def cartesian(x,n): 
    return np.vstack(np.array([np.array(np.meshgrid(*i)).T.reshape(-1,n) for i in x.values]))

In [7]:
villages_shp.head()
#villages_shp.shape

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry,RATE
0,Z,09007,連江縣,Lienchiang County,(POLYGON ((119.9644519140001 25.94552317400007...,13.32
1,G,10002,宜蘭縣,Yilan County,(POLYGON ((121.9597084550001 24.84493697000005...,17.10
2,N,10007,彰化縣,Changhua County,"POLYGON ((120.4565526600001 24.20718620500008,...",15.91
3,M,10008,南投縣,Nantou County,"POLYGON ((121.2708644380001 24.23660985400005,...",15.08
4,P,10009,雲林縣,Yunlin County,"(POLYGON ((120.081077282 23.52412216400006, 12...",14.35


In [8]:
villages_shp = gpd.GeoDataFrame(cartesian(villages_shp, 5+1),columns=villages_shp.columns)

In [9]:
villages_shp.head(6)

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry,RATE
0,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9644519140001 25.94552317400007,...",13.32
1,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9646932640001 25.94687440200005,...",13.32
2,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9643200390001 25.94638165700007,...",13.32
3,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9848039630001 25.96429508900007,...",13.32
4,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9853612840001 25.96461429900006,...",13.32
5,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9893897520001 25.97037512200006,...",13.32


In [10]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [11]:
villages_shp['x'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
villages_shp['y'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [12]:
villages_shp.head(6)

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry,RATE,x,y
0,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9644519140001 25.94552317400007,...",13.32,"[119.96445191400005, 119.96426754100003, 119.9...","[25.945523174000073, 25.945488368000042, 25.94..."
1,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9646932640001 25.94687440200005,...",13.32,"[119.96469326400006, 119.96469944500005, 119.9...","[25.94687440200005, 25.946863776000043, 25.946..."
2,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9643200390001 25.94638165700007,...",13.32,"[119.96432003900009, 119.96411690100001, 119.9...","[25.946381657000074, 25.946294144000035, 25.94..."
3,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9848039630001 25.96429508900007,...",13.32,"[119.9848039630001, 119.98479340300003, 119.98...","[25.964295089000075, 25.964292340000043, 25.96..."
4,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9853612840001 25.96461429900006,...",13.32,"[119.98536128400006, 119.98530636400007, 119.9...","[25.964614299000065, 25.964601334000065, 25.96..."
5,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9893897520001 25.97037512200006,...",13.32,"[119.9893897520001, 119.98963574700008, 119.98...","[25.970375122000064, 25.970334440000045, 25.97..."


In [13]:
town_xs = villages_shp.x
town_ys = villages_shp.y
#town_name = villages_shp.COUNTYNAME + ", " +  villages_shp.TOWNNAME
town_name = villages_shp.COUNTYNAME
town_rate = villages_shp.RATE

In [14]:
#villages_shp_source = GeoJSONDataSource(geojson=villages_shp.to_json())
source = ColumnDataSource(data=dict(
    x=town_xs,
    y=town_ys,
    name=town_name,
    rate=town_rate, 
))

TOOLTIPS = [
    ("Latitude", "$x"),
    ("Longitude", "$y"),
    ("County, Town", "@name"), 
    ("Rate", "@rate{%0.2f}%")
]
color_mapper = LogColorMapper(palette=palette)# Color map
#HoverTool.mode="vline"

In [15]:
# Initialize our plot figure
p = figure(title="Taiwan Map", plot_width=1200, plot_height=900,
          tools="pan,wheel_zoom,box_zoom,reset")
p.axis.visible=False
p.add_tools(HoverTool(
    tooltips=TOOLTIPS,
    
    formatters={
        'rate' : 'printf',   # use 'printf' formatter for 'rate' field
                             # use default 'numeral' formatter for other fields
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    #mode='vline'
))

p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

zoom_overlay = p.select_one(BoxZoomTool).overlay

zoom_overlay.line_color = "olive"
zoom_overlay.line_width = 5
zoom_overlay.line_dash = "solid"
zoom_overlay.fill_color = None

In [16]:
# Add the lines to the map from our GeoJSONDataSource -object
#p.multi_line('x', 'y', source=source, color='black', line_width=2)
p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.3)

GlyphRenderer(id='1048', ...)

In [17]:
# add callback to control 
callback = CustomJS(args=dict(p=p, source=source), code="""

            var radio_value = cb_obj.get('value');
            var data = source.data; 
            rate = data['rate']

            for (i = 0; i < rate.length; i++) {
                if(rate[i] >= radio_value) {
                    rate[i] = rate[i];
                } else {
                    rate[i] = undefined;
                }
            }
        source.change.emit();
        """)

# option
option = RadioButtonGroup(labels=["20", "30", "40"],
                          active=0, callback=callback)

ValueError: expected an element of List(String), got seq with invalid items [20, 30, 40]

In [ ]:
output_file("taiwanmap.html")
#output_notebook()

In [ ]:
#show(p)
save(row(p, option), title="taiwanmap")

In [ ]:
#source.data['x']